<a href="https://colab.research.google.com/github/ardy-dl/7_11_recreate/blob/main/ardy-dl%20/%20big_data_manipulationsManipulation_as_RDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Operation cancelled by user


In [ ]:
from pyspark import SparkContext

In [ ]:
spark.stop()

In [ ]:
sc = pyspark.SparkContext(appName="performanceFactors")

In [ ]:
factors = sc.textFile("/content/StudentPerformanceFactors.csv")

In [ ]:
factors.getNumPartitions()

2

In [ ]:
print(type(factors))

<class 'pyspark.rdd.RDD'>


In [ ]:
print(factors.getNumPartitions())

2


In [2]:
!pip install pyspark
from pyspark import SparkContext
sc = SparkContext(appName="performanceFactors")
factorsrdd = sc.textFile("/content/StudentPerformanceFactors.csv")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=b46ce8067aa3b5df4eb8653846808c0a8c0d6bc414a7fb0b31b6e6e4a5479561
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [44]:
header = factorsrdd.first()  # header
datardd = factorsrdd.filter(lambda row: row != header).map(lambda line: line.split(","))  # rdd

column_names = header.split(",")  # split headers
column_index = {column_names[i].strip(): i for i in range(len(column_names))}  # dictionary for each column

# index of specific columns
exam_score_column = column_index['Exam_Score']
previous_exam_column = column_index['Previous_Scores']
sleep_column = column_index['Sleep_Hours']
hrs_studied_column = column_index['Hours_Studied']
income_column = column_index['Family_Income']

# Filter students who passed both exams
passed_both_filtered = datardd.filter(lambda row: (
    int(row[previous_exam_column].strip()) >= 70 and
    int(row[exam_score_column].strip()) >= 70
))

# Filter students who passed the previous exam but not the recent one
passed_previous_filtered = datardd.filter(lambda row: (
    int(row[previous_exam_column].strip()) >= 70 and
    int(row[exam_score_column].strip()) < 70
))

# Combine selected columns
both_exam = passed_both_filtered.map(lambda row: (
    int(row[hrs_studied_column].strip()),
    (int(row[previous_exam_column].strip()),
     row[income_column].strip(),
     row[sleep_column].strip() + " hrs",
     int(row[exam_score_column].strip()))
))

previous_exam = passed_previous_filtered.map(lambda row: (
    row[income_column].strip(),
    (int(row[previous_exam_column].strip()),
     int(row[hrs_studied_column].strip()),
     row[sleep_column].strip() + " hrs",
     int(row[exam_score_column].strip()))
))

# Group by Family Income and sort by Previous Exam Score within each group
groupby_income = previous_exam.groupByKey()
sortedby_income = groupby_income.mapValues(lambda records: sorted(records, key=lambda x: x[1]))

# Collect and print the results
groupsorted_data = sortedby_income.collect()

for group_income, records in groupsorted_data:
    print("Family Income: ", group_income)
    for record in records:
        print(f"  Hours_Studied: {record[1]}, Previous_Score: {record[0]}, Sleep_Hours: {record[2]}, Recent_Exam_Score: {record[3]}")


Family Income:  Low
  Hours_Studied: 1, Previous_Score: 97, Sleep_Hours: 6 hrs, Recent_Exam_Score: 61
  Hours_Studied: 2, Previous_Score: 80, Sleep_Hours: 7 hrs, Recent_Exam_Score: 66
  Hours_Studied: 2, Previous_Score: 73, Sleep_Hours: 6 hrs, Recent_Exam_Score: 58
  Hours_Studied: 3, Previous_Score: 73, Sleep_Hours: 8 hrs, Recent_Exam_Score: 62
  Hours_Studied: 3, Previous_Score: 78, Sleep_Hours: 6 hrs, Recent_Exam_Score: 57
  Hours_Studied: 4, Previous_Score: 71, Sleep_Hours: 7 hrs, Recent_Exam_Score: 58
  Hours_Studied: 4, Previous_Score: 71, Sleep_Hours: 8 hrs, Recent_Exam_Score: 64
  Hours_Studied: 4, Previous_Score: 84, Sleep_Hours: 8 hrs, Recent_Exam_Score: 60
  Hours_Studied: 4, Previous_Score: 99, Sleep_Hours: 7 hrs, Recent_Exam_Score: 64
  Hours_Studied: 4, Previous_Score: 97, Sleep_Hours: 5 hrs, Recent_Exam_Score: 60
  Hours_Studied: 4, Previous_Score: 86, Sleep_Hours: 4 hrs, Recent_Exam_Score: 64
  Hours_Studied: 5, Previous_Score: 98, Sleep_Hours: 7 hrs, Recent_Exam_Score:

In [20]:
sc.stop()

In [ ]:
highest_exam = exam_scores.max()
lowest_exam = exam_scores.min()

print("Lowest Exam Score:", lowest_exam)
print("Highest Exam Score:", highest_exam)